In [11]:
from policyengine_us import Microsimulation
import pandas as pd
from microdf import MicroDataFrame

cps_sim = Microsimulation(dataset="cps_2023")
enhanced_cps_sim = Microsimulation(dataset="enhanced_cps")

puf = MicroDataFrame(
    pd.read_csv("puf_full_demographics.csv.gz"), weights="decimal_weight"
)

In [5]:
parameters = cps_sim.tax_benefit_system.parameters("2023-01-01").calibration
import numpy as np

agi_thresholds = list(
    parameters.gov.irs.soi.agi.number_of_returns.thresholds
) + [np.inf]
num_returns = parameters.gov.irs.soi.agi.number_of_returns.amounts

import plotly.express as px

band_names = []
sources = []
values = []

cps_agi = cps_sim.calculate("adjusted_gross_income")
enhanced_cps_agi = enhanced_cps_sim.calculate("adjusted_gross_income")
puf_agi = puf.adjusted_gross_income
for i in range(len(agi_thresholds[:-1])):
    band_name = f"{agi_thresholds[i]:,.0f}-{agi_thresholds[i+1]:,.0f}"

    band_names.extend([band_name] * 3)
    sources.extend(["CPS", "Enhanced CPS", "PUF (2015)"])

    cps_in_band = cps_agi.between(agi_thresholds[i], agi_thresholds[i + 1])
    enhanced_cps_in_band = enhanced_cps_agi.between(
        agi_thresholds[i], agi_thresholds[i + 1]
    )
    puf_in_band = puf_agi.between(agi_thresholds[i], agi_thresholds[i + 1])
    target = num_returns[i]

    values.extend(
        [
            cps_agi[cps_in_band].count() / target - 1,
            enhanced_cps_agi[enhanced_cps_in_band].count() / target - 1,
            puf_agi[puf_in_band].count() / target - 1,
        ]
    )
    # values = list(map(abs, values))

df = pd.DataFrame(
    {"Band": band_names, "Source": sources, "Number of returns error": values}
)

df["Number of returns error"] /= 1

from policyengine_core.charts import format_fig, BLUE, GRAY, DARK_GRAY

fig = px.bar(
    df,
    x="Band",
    y="Number of returns error",
    color="Source",
    barmode="group",
    color_discrete_map={
        "CPS": GRAY,
        "PUF (2015)": DARK_GRAY,
        "Enhanced CPS": BLUE,
    },
)

fig.update_layout(
    title="Number of returns error by dataset and AGI band",
    yaxis_title="Number of returns error (thousands)",
    yaxis_tickformat=".0%",
)

format_fig(fig)

In [10]:
parameters = cps_sim.tax_benefit_system.parameters("2023-01-01").calibration
import numpy as np

agi_thresholds = list(parameters.gov.irs.soi.agi.total_agi.thresholds) + [
    np.inf
]
num_returns = parameters.gov.irs.soi.agi.total_agi.amounts

import plotly.express as px

band_names = []
sources = []
values = []

cps_agi = cps_sim.calculate("adjusted_gross_income")
enhanced_cps_agi = enhanced_cps_sim.calculate("adjusted_gross_income")
puf_agi = puf.adjusted_gross_income
for i in range(len(agi_thresholds[:-1])):
    band_name = f"{agi_thresholds[i]:,.0f}"

    band_names.extend([band_name] * 3)
    sources.extend(["CPS", "Enhanced CPS", "PUF (2015)"])

    cps_in_band = cps_agi.between(agi_thresholds[i], agi_thresholds[i + 1])
    enhanced_cps_in_band = enhanced_cps_agi.between(
        agi_thresholds[i], agi_thresholds[i + 1]
    )
    puf_in_band = puf_agi.between(agi_thresholds[i], agi_thresholds[i + 1])
    target = num_returns[i]

    values.extend(
        [
            cps_agi[cps_in_band].sum() / target - 1,
            enhanced_cps_agi[enhanced_cps_in_band].sum() / target - 1,
            puf_agi[puf_in_band].sum() / target - 1,
        ]
    )

    # values = list(map(abs, values))

df = pd.DataFrame(
    {"Band": band_names, "Source": sources, "Total AGI error": values}
)

df["Total AGI error"] /= 1

from policyengine_core.charts import format_fig, BLUE, GRAY, DARK_GRAY

fig = px.bar(
    df,
    x="Band",
    y="Total AGI error",
    color="Source",
    barmode="group",
    color_discrete_map={
        "CPS": GRAY,
        "PUF (2015)": DARK_GRAY,
        "Enhanced CPS": BLUE,
    },
)

fig.update_layout(
    title="Total AGI error by dataset and AGI band",
    yaxis_title="Total AGI error ($bn)",
    yaxis_tickformat="+.0%",
)

format_fig(fig)

In [7]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
from policyengine_core.periods import instant
import pandas as pd


def modify_parameters(parameters):
    parameters.gov.irs.income.bracket.rates.children["7"].update(
        start=instant("2023-01-01"), stop=instant("2028-12-31"), value=0.5
    )
    return parameters


class reform(Reform):
    def apply(self):
        self.modify_parameters(modify_parameters)


baseline = Microsimulation()
reformed = Microsimulation(reform=reform)
HOUSEHOLD_VARIABLES = [
    "person_id",
    "household_id",
    "age",
    "household_net_income",
    "adjusted_gross_income",
    "employment_income",
    "household_income_decile",
    "in_poverty",
    "household_tax",
    "household_benefits",
]
baseline_person_df = baseline.calculate_dataframe(
    HOUSEHOLD_VARIABLES, 2023
).astype(float)
reformed_person_df = reformed.calculate_dataframe(
    HOUSEHOLD_VARIABLES, 2023
).astype(float)
difference_person_df = reformed_person_df - baseline_person_df

In [ ]:
HOUSEHOLD_VARIABLES = [
    "person_id",
    "household_id",
    "age",
    "household_net_income",
    "adjusted_gross_income",
    "employment_income",
    "household_income_decile",
    "in_poverty",
    "household_tax",
    "household_benefits",
]
baseline_person_df = baseline.calculate_dataframe(
    HOUSEHOLD_VARIABLES, 2023
).astype(float)
reformed_person_df = reformed.calculate_dataframe(
    HOUSEHOLD_VARIABLES, 2023
).astype(float)
difference_person_df = reformed_person_df - baseline_person_df

baseline_person_df["gain"] = difference_person_df["household_net_income"]

In [ ]:
baseline_person_df[baseline_person_df.gain < 0][
    baseline_person_df.household_net_income < 0
]

,person_id,household_id,age,household_net_income,household_income_decile,in_poverty,household_tax,household_benefits,gain
146135,100000401.0,100000000.0,52.0,-861366.75000,1.0,1.0,2.964205e+06,0.0,-770664.500
146136,100000402.0,100000000.0,51.0,-861366.75000,1.0,1.0,2.964205e+06,0.0,-770664.500
146138,100001601.0,100000016.0,65.0,-708258.25000,1.0,1.0,1.596674e+06,43562.0,-389161.750
146139,100001602.0,100000016.0,68.0,-708258.25000,1.0,1.0,1.596674e+06,43562.0,-389161.750
146144,100002201.0,100000024.0,63.0,-681460.75000,1.0,1.0,1.072878e+06,0.0,-233571.750
...,...,...,...,...,...,...,...,...,...
292105,108893004.0,100088928.0,28.0,-500629.75000,1.0,1.0,1.104709e+06,40867.0,-299445.000
292106,108893005.0,100088928.0,85.0,-500629.75000,1.0,1.0,1.104709e+06,40867.0,-299445.000
292107,108893006.0,100088928.0,75.0,-500629.75000,1.0,0.0,1.104709e+06,40867.0,-299445.000
292114,108893401.0,100088936.0,58.0,-32869.34375,1.0,1.0,4.075813e+05,0.0,-65146.625
